In [17]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import os
import re
import copy
import json

In [18]:
PIECES_TO_NUM = {
    "square": 0,
    "white_pawn": 1,
    "white_rook": 2,
    "white_knight": 3,
    "white_bishop": 4,
    "white_king": 5,
    "white_queen": 6,
    "white_unknown": 7,
    "black_pawn": -1,
    "black_rook": -2,
    "black_knight": -3,
    "black_bishop": -4,
    "black_king": -5,
    "black_queen": -6,
    "black_unknown": -7,
}

NUM_TO_PIECE = {
    0: "square",
    1: "white_pawn",
    2: "white_rook",
    3: "white_knight",
    4: "white_bishop",
    5: "white_king",
    6: "white_queen",
    7: "white_unknown",
    -1: "black_pawn",
    -2: "black_rook",
    -3: "black_knight",
    -4: "black_bishop",
    -5: "black_king",
    -6: "black_queen",
    -7: "black_unknown"
}

In [19]:
piece_images = {
    piece: Image.open(f'assets/{piece}.png') for piece in PIECES_TO_NUM.keys() if piece != "square"
}

In [20]:
game_state = np.array([
    [-2, -3, -4, -6, -5, -4, -3, -2],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  -7],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  7],
    [ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 2,  3,  4,  6,  5,  4,  3,  2],
])

def draw_chessboard(board, save=False):
    board = np.flipud(board)
    fig, ax = plt.subplots()

    chessboard_pattern = np.zeros((8, 8))
    chessboard_pattern[1::2, ::2] = 1
    chessboard_pattern[::2, 1::2] = 1

    ax.imshow(chessboard_pattern, cmap='gray', interpolation='none')

    for i in range(8):
        for j in range(8):
            piece_value = board[i, j]
            if piece_value != 0:
                piece_name = NUM_TO_PIECE[piece_value]
                piece_image = piece_images[piece_name]
                ax.imshow(piece_image, extent=[j - 0.5, j + 0.5, i - 0.5, i + 0.5])

    ax.set_xticks(np.arange(8))
    ax.set_yticks(np.arange(8))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks(np.arange(-0.5, 8, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 8, 1), minor=True)
    ax.grid(which="minor", color="black", linestyle='-', linewidth=2)

    if save:
        fig.savefig(f"{save}.png")

    # plt.show()
    matplotlib.pyplot.close()
    
draw_chessboard(game_state)

In [21]:
def modify_game_state(game_state, move):
    new_game_state = copy.copy(np.flipud(game_state)) # A1 is 0,0
    if " -> " in move:
        init_pos = move.split(" -> ")[0]
        new_pos = move.split(" -> ")[1]

        init_y_pos = int(init_pos[-1]) - 1
        init_x_pos = int(ord(init_pos[-2]) - 96 - 1)

        new_y_pos = int(new_pos[-1]) - 1
        new_x_pos = int(ord(new_pos[-2]) - 96 - 1)      

        piece = new_game_state[init_y_pos, init_x_pos]
        new_game_state[init_y_pos, init_x_pos] = 0
        new_game_state[new_y_pos, new_x_pos] = piece

    elif " + " in move:
        init_pos = move.split(" + ")[0]
        new_pos = move.split(" + ")[1]

        init_y_pos = int(init_pos[-1]) - 1
        init_x_pos = int(ord(init_pos[-2]) - 96 - 1)

        new_y_pos = int(new_pos[-1]) - 1
        new_x_pos = int(ord(new_pos[-2]) - 96 - 1)      

        piece = new_game_state[init_y_pos, init_x_pos]
        new_game_state[init_y_pos, init_x_pos] = 0
        new_game_state[new_y_pos, new_x_pos] = piece

    elif " x " in move:
        init_pos = move.split(" x ")[0]
        new_pos = move.split(" x ")[1]

        init_y_pos = int(init_pos[-1]) - 1
        init_x_pos = int(ord(init_pos[-2]) - 96 - 1)

        new_y_pos = int(new_pos[-1]) - 1
        new_x_pos = int(ord(new_pos[-2]) - 96 - 1)      

        piece = new_game_state[init_y_pos, init_x_pos]
        new_game_state[init_y_pos, init_x_pos] = 0
        new_game_state[new_y_pos, new_x_pos] = piece

    #prise en passant 
    elif " @ " in move:
        init_pos = move.split(" @ ")[0]
        new_pos = move.split(" @ ")[1]

        init_y_pos = int(init_pos[-1]) - 1
        init_x_pos = int(ord(init_pos[-2]) - 96 - 1)

        new_y_pos = int(new_pos[-1]) - 1
        new_x_pos = int(ord(new_pos[-2]) - 96 - 1)      

        piece = new_game_state[init_y_pos, init_x_pos]
        new_game_state[init_y_pos, init_x_pos] = 0
        new_game_state[init_y_pos, init_x_pos+1] = 0
        new_game_state[new_y_pos, new_x_pos] = piece

    elif "O-O-O" in move:
        if move[0] == "w":
            new_game_state[0, 2] = new_game_state[0, 4]
            new_game_state[0, 3] = new_game_state[0, 0]
            new_game_state[0, 0] = 0
            new_game_state[0, 4] = 0
        else:
            new_game_state[7, 2] = new_game_state[7, 4]
            new_game_state[7, 3] = new_game_state[7, 0]
            new_game_state[7, 0] = 0
            new_game_state[7, 4] = 0

    elif "O-O" in move:
        if move[0] == "w":
            new_game_state[0, 6] = new_game_state[0, 4]
            new_game_state[0, 5] = new_game_state[0, 7]
            new_game_state[0, 7] = 0
            new_game_state[0, 4] = 0
        else:
            new_game_state[7, 6] = new_game_state[7, 4]
            new_game_state[7, 5] = new_game_state[7, 7]
            new_game_state[7, 7] = 0
            new_game_state[7, 4] = 0
        

    return np.flipud(new_game_state)

In [40]:
# "->" is a move
# "x" is a capture
# "+" is check
# "#" is checkmate
# "O-O" is kingside castle
# "O-O-O" is queenside castle

# fix-game LR
game_fix = [
    "e2 -> e4",
    "e7 -> e5",
    "Ng1 -> Nf3",
    "Nb8 -> Nc6",
    "Bf1 -> Bb5",
    "a7 -> a6",
    "Bb5 -> Ba4",
    "Ng8 -> Nf6",
    "wO-O",
    "Bf8 -> Be7",
    "Rf1 -> Re1",
    "b7 -> b5",
    "Ba4 -> Bb3",
    "d7 -> d6",
    "c2 -> c3",
    "O-O",
    "h2 -> h3",
    "Bc8 -> Be6",
    "d2 -> d4",
    "Be6 x Bb3",
    "a2 x Bb3",
    "e5 x d4",
    "c3 x d4",
    "d6 -> d5",
    "e4 -> e5",
    "Nf6 -> Ne4",
    "Nb1 -> Nc3",
    "f7 -> f5",
    # prise en passant
    "e5 @ f6",
    "Ne4 x f6",
    "Bc1 -> Bg5",
    "Nc6 -> Nb4",
    "Nc3 x b5",
    "Qd8 -> Qd7",
    "Nb5 -> Nc3",
    "Be7 -> Bd6",
    "Re1 -> Re3",
    "h7 -> h6",
    "Bg5 x Nf6",
    "Rf8 x Bf6",
    "Ra1 -> Ra5",
    "Ra8 -> Rf8",
    "Nc3 -> Na2",
    "Rf6 -> Re6",
    "Nf3 -> Ne5",
    "Bd6 x Ne5",
    "Na2 x Nb4",
    "Be5 -> Bd6",
    "Nb4 x d5",
    "Rf8 -> Rf5",
    "b3 -> b4",
    "Bd6 x b4",
    "Re3 x Re6",
    "Bb4 x Ra5",
    "Nd5 -> Ne7",
    "Kg8 -> Kf7",
    "Re6 -> Re3",
    "Qd7 x Ne7",
    "Re3 + Qe7",
    "Kf7 x Re7",
    "Qd1 + Qe2",
    "Ke7 -> Kf7",
    "Qe2 + Qc4",
    "Kf7 -> Kg6",
    "Qc4 + Qe6",
    "Rf5 -> Rf6",
    "Qe6 + Qe4", 
]

game_mov = [
    "d2 -> d4",
    "Ng8 -> Nf6",
    "c2 -> c4",
    "g7 -> g6",
    "Nb1 -> Nc3",
    "Bf8 -> Bg7",
    "e2 -> e4",
    "d7 -> d6",
    "Ng1 -> Nf3",
    "bO-O",
    "Bf1 -> Be2",
    "e7 -> e5",
    "d4 -> d5",
    "a7 -> a5",
    "Bc1 -> Bg5",
    "h7 -> h6",
    "Bg5 -> Bh4",
    "Nb8 -> Na6",
    "wO-O",
    "Qd8 -> Qe8",
    "Nf3 -> Nd2",
    "Nf6 -> Nh7",
    "a2 -> a3",
    "Bc8 -> Bd7",
    "b2 -> b3",
    "Bg7 -> Bf6",
    "Bh4 x Bf6",
    "Nh7 x Bf6",
    "Qd1 -> Qc2",
    "b7 -> b6",
    "Ra1 -> Rb1",
    "Nf6 -> Nh7",
    "b3 -> b4",
    "f7 -> f5",
    "e4 x f5",
    "g6 x f5",
    "Kg1 -> Kh1",
    "Qe8 -> Qg6",
    "f2 -> f4",
    "Kg8 -> Kh8",
    "f4 x e5",
    "d6 x e5",
    "g2 -> g4",
    "a5 x b4",
    "a3 x b4",
    "c7 -> c5",
    "b4 x c5",
    "Na6 x c5",
    "Nd2 -> Nb3",
    "Rf8 -> Rg8",
    "Nb3 x Nc5",
    "b6 x Nc5",
    "Qc2 -> Qd3",
    "h6 -> h5",
    "Rb1 -> Rb7",
    "Nh7 -> Nf6",
    "Qd3 -> Qe3",
    "Ra8 -> Re8",
    "Qe3 x c5",
    "h5 x g4",
    "Qc5 -> Qf2",
    "Qg6 -> Qh6",
    "Rf1 -> Rg1",
    "f5 -> f4",
    "Be2 -> Bd3",
    "g4 -> g3",
    "Qf2 -> Qb2",
    "f4 -> f3",
    "Rb7 -> Rb6",
    "g3 + g2"
]

frames_fix = {
    0: 48,
    1: 122,
    2: 207,
    3: 363,
    4: 487,
    5: 638,
    6: 741,
    7: 826,
    8: 963,
    9: 1178,
    10: 1293,
    11: 1368,
    12: 1474,
    13: 1565,
    14: 1658,
    15: 1738,
    16: 1859,
    17: 1923,
    18: 2062,
    19: 2170,
    20: 2355,
    21: 2521,
    22: 2669,
    23: 2801,
    24: 2891,
    25: 2981,
    26: 3126,
    27: 3221,
    28: 3298,
    29: 3493,
    30: 3658,
    31: 3787,
    32: 3884,
    33: 4008,
    34: 4081,
    35: 4273,
    36: 4368,
    37: 4458,
    38: 4525,
    39: 4677,
    40: 4781,
    41: 4897,
    42: 4977,
    43: 5066,
    44: 5143,
    45: 5245,
    46: 5421,
    47: 5530,
    48: 5677,
    49: 5850,
    50: 5949,
    51: 6043,
    52: 6169,
    53: 6320,
    54: 6444,
    55: 6579,
    56: 6674,
    57: 6794,
    58: 6895,
    59: 7089,
    60: 7217,
    61: 7289,
    62: 7375,
    63: 7486,
    64: 7577,
    65: 7665,
    66: 7752,
    67: 7824
}

frames_mov = {
    0: 26,
    1: 121,
    2: 249,
    3: 316,
    4: 392,
    5: 508,
    6: 580,
    7: 694,
    8: 775,
    9: 909,
    10: 1053,
    11: 1130,
    12: 1245,
    13: 1318,
    14: 1415,
    15: 1533,
    16: 1613,
    17: 1723,
    18: 1882,
    19: 2011,
    20: 2113,
    21: 2244,
    22: 2392,
    23: 2482,
    24: 2599,
    25: 2682,
    26: 2818,
    27: 2986,
    28: 3175,
    29: 3265,
    30: 3343,
    31: 3413,
    32: 3556,
    33: 3631,
    34: 3761,
    35: 3909,
    36: 4003,
    37: 4108,
    38: 4226,
    39: 4365,
    40: 4444,
    41: 4661,
    42: 4788,
    43: 4884,
    44: 5055,
    45: 5165,
    46: 5281,
    47: 5420,
    48: 5526,
    49: 5686,
    50: 5772,
    51: 6003,
    52: 6144,
    53: 6284,
    54: 6381,
    55: 6541,
    56: 6677,
    57: 6759,
    58: 6839,
    59: 7018,
    60: 7145,
    61: 7259,
    62: 7354,
    63: 7425,
    64: 7532,
    65: 7604,
    66: 7684,
    67: 7777,
    68: 7868,
    69: 7987,
    70: 8111
}




In [41]:
data = {} # data contains the list of the move and the game_state + frame_number after each move
data["moves"] = game_fix
# data["moves"] = game_mov
data["game_states"] = []

In [42]:
initial_game_state = np.array([
    [-2, -3, -4, -6, -5, -4, -3, -2],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 2,  3,  4,  6,  5,  4,  3,  2],
])

#### GAME_FIXED
# We always 
data["game_states"].append({"frame": frames_fix.get(0),
                            "gs": initial_game_state.tolist()
                           })
draw_chessboard(initial_game_state, "step1_Lea/game_fix/gs_0")

new_gs = modify_game_state(initial_game_state, game_fix[0])

# In you case, the frame number won't be 1, this is just an example here.
data["game_states"].append({"frame": frames_fix.get(1),
                            "gs": new_gs.tolist()
                           })
draw_chessboard(new_gs, "step1_Lea/game_fix/gs_1")

j=2
for i, g in enumerate(game_fix[1:]):
    new_gs = modify_game_state(new_gs, g)
    data["game_states"].append({"frame": frames_fix.get(j),
                            "gs": new_gs.tolist()
                           })
    draw_chessboard(new_gs, f"step1_Lea/game_fix/gs_{j}")
    j += 1

#### GAME_MOVE
# # We always 
# data["game_states"].append({"frame": frames_mov.get(0),
#                             "gs": initial_game_state.tolist()
#                            })
# draw_chessboard(initial_game_state, "step1_Lea/game_mov/gs_0")

# new_gs = modify_game_state(initial_game_state, game_mov[0])

# # In you case, the frame number won't be 1, this is just an example here.
# data["game_states"].append({"frame": frames_mov.get(1),
#                             "gs": new_gs.tolist()
#                            })
# draw_chessboard(new_gs, "step1_Lea/game_mov/gs_1")

# j = 2
# for i, g in enumerate(game_mov[1:]):
#     new_gs = modify_game_state(new_gs, g)
#     data["game_states"].append({"frame": frames_mov.get(j),
#                             "gs": new_gs.tolist()
#                            })
#     draw_chessboard(new_gs, f"step1_Lea/game_mov/gs_{j}")
#     j += 1

In [43]:
# save the data in a json file
json.dump(data, open("step1_Lea/gameLR_fix.json", "w"))
# json.dump(data, open("step1_Lea/gameLR_mov.json", "w"))

In [44]:
initial_game_state[0, :] = -7
initial_game_state[1, :] = -7

initial_game_state[6, :] = 7
initial_game_state[7, :] = 7

draw_chessboard(initial_game_state, "step1_Lea/unknown_fix/gs_0")
# draw_chessboard(initial_game_state, "step1_Lea/unknown_mov/gs_0")

new_gs = modify_game_state(initial_game_state, game_fix[0])
draw_chessboard(new_gs, "step1_Lea/unknown_fix/gs_1")
# new_gs = modify_game_state(initial_game_state, game_mov[0])
# draw_chessboard(new_gs, "step1_Lea/unknown_mov/gs_1")

for i, g in enumerate(game_fix[1:]):
# for i, g in enumerate(game_mov[1:]):
    new_gs = modify_game_state(new_gs, g)
    # draw_chessboard(new_gs, f"step1_Lea/unknown_mov//gs_{i+2}")
    draw_chessboard(new_gs, f"step1_Lea/unknown_fix//gs_{i+2}")

In [45]:
# You can retrieve the game state from the json file
# but don't forget to convert the list to a numpy array
draw_chessboard(np.array(data["game_states"][0]["gs"]))